In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#%cd /content/gdrive/My\ Drive/Colab/IDL_Final_Project/new
%cd /content/gdrive/My\ Drive/project/IDL_Final_Project/new
#%cd /content/gdrive/My\ Drive/IDL_Final_Project/new

/content/gdrive/.shortcut-targets-by-id/1cLTX5b-YNPmc2AuVB16CsNFPAABZZ53P/IDL_Final_Project/new


In [3]:
import pandas as pd
import librosa
import numpy as np
import IPython.display as ipd
import scipy

from scipy.io import wavfile
import scipy.signal
from time import time
import matplotlib.pyplot as plt
import warnings
import os
%matplotlib inline

In [ ]:
ipd.Audio('mixer/wav_files/20090713_122937_5.wav')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
ipd.Audio('mixer/1-channel-wav/20100310_124750_4615.wav')

Output hidden; open in https://colab.research.google.com to view.

## Removing Noise

In [ ]:
waveform, sr = librosa.load('mixer/wav_files/20090713_122937_5.wav')

In [ ]:
# wav_loc = 'mixer/wav_files/20090713_122937_5.wav'
# src_rate, src_data = wavfile.read(wav_loc)
# # src_data = np.concatenate((src_data, np.zeros(src_rate*3)))
# src_data = src_data / 32768
# wav_loc = "assets/audio/cafe.wav"
# noise_rate, noise_data = wavfile.read(wav_loc)
# # get some noise to add to the signal
# noise_to_add = noise_data[len(src_data) : len(src_data) * 2]
# # get a different part of the noise clip for calculating statistics
# noise_clip = noise_data[: len(src_data)]
# noise_clip = noise_clip / max(noise_to_add)
# noise_to_add = noise_to_add / max(noise_to_add)
# # apply noise
# snr = 1  # signal to noise ratio
# audio_clip_cafe = src_data + noise_to_add / snr
# noise_clip = noise_clip / snr

## Extracting Features from Audio

In [4]:
# 1000_downloaded_mixer6_calls.csv
df_subjs = pd.read_csv("mixer/mx6_subjs.csv")
df_subjs.head(n=2)

,subjid,sex,yob,edu_years,edu_degree,edu_deg_yr,edu_contig,esl_age,ntv_lg,oth_lgs,occup,cntry_born,state_born,city_born,cntry_rsd,state_rsd,city_rsd,ethnic,smoker,ht_cm,wt_kg,mother_born,mother_raised,mother_lang,mother_edu,father_born,father_raised,father_lang,father_edu
0,103187,F,1984.0,13.0,NaN,NaN,NaN,15.0,CHI,ENG,Student,France,Hauts-de-Seine,Courbevoie,France,NaN,Gennevilliers,Asian,N,152.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,104050,F,1975.0,17.0,NaN,NaN,NaN,10.0,RUS,ENG,part-time linguist,Russia,AL,Moscow,Russia,AL,Moscow,NaN,N,164.0,61.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_calls = pd.read_csv("mixer/mx6_calls.csv")
df_calls.head(n=2)

,call_id,call_date,lang,eng_stat,sid_a,phid_a,ph_categ_a,phtyp_a,phmic_a,cnvq_a,sigq_a,tbug_a,sid_b,phid_b,ph_categ_b,phtyp_b,phmic_b,cnvq_b,sigq_b,tbug_b,topic
0,4,2009-07-13_09:46:22,ENG,All_ENG,120304,215285bro,O,1.0,4.0,G,G,N,120209,215898rjj,O,3.0,4.0,NaN,NaN,NaN,39
1,5,2009-07-13_12:25:59,ENG,All_ENG,120245,215285bro,O,1.0,4.0,G,G,N,120284,215898rjj,M,3.0,4.0,NaN,NaN,NaN,39


In [6]:
df_meta = pd.read_csv('mixer/1000_downloaded_mixer6_calls.csv')
df_meta.head(n=2)

,filename,call_date,call_id,subject_id,yob,sex,ht_cm
0,20090713_122937_5.wav,2009-07-13_12:25:59,5,120245,1985.0,M,178.0
1,20090714_121333_11.wav,2009-07-14_12:10:00,11,120263,1982.0,M,170.0


### Extracting Features

In [7]:
def mel_spectrogram(audio, n_mels,fmin,fmax):
  y, sr = librosa.load(audio)
  ftt = np.abs(librosa.stft(y, n_fft=100,  hop_length=90))**2
  mel = librosa.filters.mel(sr, 100, n_mels= n_mels, fmin = fmin, fmax= fmax)


  # mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=n_fft, hop_length=hop_length,
  #                                      n_mels=n_mels, fmin = fmin,fmax=fmax)

  return mel.dot(ftt)


In [8]:
def extract_mel_spectrograms(filename):
  ''' 
  filename - file to parse
  TODO: Implement this method to extract mel spectrograms of filename

  Returns a numpy array of extract data
  '''
  #mixer/wave_file
  # TODO
  #for audio in filename:
  #mixer6/data/ulaw_sphere/20090713_122937_5.wav
  mel_Spectrogram = mel_spectrogram(filename, 64,150,3800)
  
  return mel_Spectrogram

In [9]:
def extract_speaker_data(dir,key):
  # dir = 'train/dr6/fbch0'
  audio_filnames = dir#get_filenames(dir)

  dev_data = []
  for audio_filename in audio_filnames:
    au = str(key[0])+'/'+ str(key[1])+'/'+audio_filename
    #print('au',au)
    mel = extract_mel_spectrograms(au)
    #break
    dev_data.append(mel)

  return dev_data

In [ ]:
warnings.filterwarnings(action='once')  # Suprressing warnings

data = []
labels = []
gender = {'M': 1, 'F': 0,}

for row in df_meta[df_meta.yob.notna()].itertuples():
  start = time()
  #print(row.filename)
  # filename = "mixer/wav_files/"+ row.filename
  addition = row.filename.split('.')
  addition = str(addition[0])
  directory = "new_1/"+ addition #"mixer/1-channel-wav/"+ row.filename
  #print(directory)
  #break
  for root, dirs, files in os.walk(directory):
    path_keys = root.split('/')
    '''print(path_keys)
    print(dirs)
    print(files)'''
    try:
        feat = extract_speaker_data(files,path_keys)
        data.extend(feat)
    except EOFError as e:
        print(e)
        print('Error occurred with file: ',root)
        continue
    
    
  #break
    rec_year = int(row.call_date.split('-')[0])  # Year a call was made
    yob = row.yob
    sex = gender[row.sex] if row.sex in gender.keys() else -1
    ht_cm = row.ht_cm

    age = rec_year - yob
    #print(age, ht_cm, sex,len(feat))
    #1. exctract mel spectrogram and append to the data array
    #feat = extract_mel_spectrograms(filename)
    #print(feat)
    #data.extend(feat)

    #2. Append to the labels the corresponding array of labels
    labels.extend([[age, ht_cm, sex]]*len(feat))
    '''print(len(data),len(labels))
    print(labels)'''

    #break
    if row.Index == 0:
      print("Time taken for one iteration: {} secs".format(time() - start))
  #break

/usr/local/lib/python3.6/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=61>
  if ffdec.available():
/usr/local/lib/python3.6/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=63>
  if ffdec.available():
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=61>
  if ffdec.available():
/usr/local/lib/python3.6/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=63>
  if ffdec.available():
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel 

Time taken for one iteration: 14.159753322601318 secs


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=61>
  if ffdec.available():
/usr/local/lib/python3.6/dist-packages/audioread/__init__.py:86: ResourceWarning: unclosed file <_io.BufferedReader name=63>
  if ffdec.available():
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/audioread/__init

In [ ]:
'''warnings.filterwarnings(action='once')  # Suprressing warnings

data = []
labels = []
gender = {'M': 1, 'F': 0,}

for row in df_meta[df_meta.yob.notna()].itertuples():
  start = time()
  #print(row.filename)
  # filename = "mixer/wav_files/"+ row.filename
  filename = "test_audio_new/"+ row.filename #"mixer/1-channel-wav/"+ row.filename
  #print(filename)
  #break
  rec_year = int(row.call_date.split('-')[0])  # Year a call was made
  yob = row.yob
  sex = gender[row.sex] if row.sex in gender.keys() else -1
  ht_cm = row.ht_cm

  age = rec_year - yob

  #1. exctract mel spectrogram and append to the data array
  feat = extract_mel_spectrograms(filename)
  #print(feat)
  data.append(feat)

  #2. Append to the labels the corresponding array of labels
  labels.append([age, ht_cm, sex])

  if row.Index == 0:
    print("Time taken for one iteration: {} secs".format(time() - start))'''


In [ ]:
# Convert lists to numpy and save numpy arrays
data_arr = np.array(data)
labels_arr = np.array(labels)

np.save('dev_data', data_arr)
np.save('dev_labels', labels_arr)

In [ ]:
len(data)